In [9]:
from src.genre_classification.F_Basic_Models import Basic_Models
from src.genre_classification.F_Dataset_Downloader import Dataset_Downloader
from src.genre_classification.F_Pretrained_models import Pretrained

import pandas as pd

In [10]:
dataset_downloader = Dataset_Downloader()
train_path, test_path = dataset_downloader()

train_data = pd.read_csv(train_path.resolve())
test_data = pd.read_csv(test_path.resolve())

train_data.columns

print(set(train_data['genre']))

{'biography', 'sci-fi', 'sport', 'short', 'adult', 'news', 'musical', 'action', 'animation', 'comedy', 'talk-show', 'thriller', 'history', 'film-noir', 'horror', 'adventure', 'war', 'mystery', 'game-show', 'music', 'fantasy', 'reality-tv', 'western', 'drama', 'romance', 'crime', 'documentary', 'family'}


In [ ]:
x_train, y_train = train_data.drop(columns=["genre"]), train_data["genre"]

x_test, y_test = test_data.drop(columns=["genre"]), test_data["genre"]

unique_labels = sorted(list(set(train_data["genre"])))


# Pruebas de Modelos Basicos

In [ ]:
basic_models_name = ['Random_Forest', 'Naive_Bayes', 'LogReg', 'Linear_SVM']

for model_name in basic_models_name:
    model = Basic_Models(model_type= model_name)
    model.fit(x_train, y_train)
    y_hat = model.transform(x_test)
    model.evaluate(y_true= y_test, y_hat= y_hat, labels = set(y_test), evaluate_type= "all_metrics")


Ha terminado el preprocesado
Ha terminado la transfromacion al embbeding
Ha terminado de entrenar
Ha terminado de preprocesar entrenamiento
Ha terminado de representar entrenamiento
(55400,) (55400,)


# Pruebas de Modelos Transformer

In [ ]:
def to_one_hot(label_str, num_classes, label_map):
    vector = [0] * num_classes
    if label_str in label_map:
        idx = label_map[label_str]
        vector[idx] = 1
    return vector

In [ ]:
train_texts = train_data["text"].tolist() 
test_texts = test_data["text"].tolist()   

id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {label: i for i, label in enumerate(unique_labels)}

train_labels_one_hot = [to_one_hot(l, len(id2label), label2id) for l in train_data["genre"]]

val = Pretrained(model_type="TOCHO", id2label=id2label) 

val.fit(train_texts, train_labels_one_hot)

predictions = val.transform(test_texts)


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

# 1. Definición del Mapeo
label_mapping = {
    # --- GRUPO 0: Action & Suspense ---
    "action": 0, "adventure": 0, "crime": 0, "fantasy": 0,
    "mystery": 0, "sci-fi": 0, "thriller": 0, "war": 0, "western": 0,

    # --- GRUPO 1: Drama & Serious ---
    "biography": 1, "drama": 1, "film-noir": 1, "history": 1, "romance": 1,

    # --- GRUPO 2: Comedy & Light ---
    "animation": 2, "comedy": 2, "family": 2, "music": 2, "musical": 2,

    # --- GRUPO 3: Factual & TV ---
    "documentary": 3, "game-show": 3, "news": 3, "reality-tv": 3,
    "sport": 3, "talk-show": 3,

    # --- GRUPO 4: Niche / Distinct ---
    "adult": 4, "horror": 4, "short": 4
}

# Nombres para el reporte
super_class_names = [
    "Action_Suspense", 
    "Drama_Serious", 
    "Comedy_Family", 
    "Factual_TV", 
    "Niche_Horror"
]

def prepare_data(df, text_col='text', label_col='genre'):
    df = df.copy()
    # CORRECCIÓN 1: Usar 'label_mapping' en lugar de 'reduction_map'
    df['label_reduced'] = df[label_col].map(label_mapping)
    
    # Limpieza extra: Eliminar filas que no se hayan podido mapear (por si acaso)
    df = df.dropna(subset=['label_reduced'])
    df['label_reduced'] = df['label_reduced'].astype(int)
    
    return df

def train_genre_classifier(train_df, text_col='text', label_col='label_reduced'):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=20000,    # Subimos de 10k a 20k para captar más vocabulario útil
            ngram_range=(1, 3),    # Unigramas y Bigramas (se mantiene)
            stop_words='english',
            min_df=5,              # NUEVO: Ignora palabras que aparecen en <5 docs (quita ruido)
            sublinear_tf=True      
        )),
        ('clf', LogisticRegression(
            multi_class='multinomial',
            solver='saga',         
            #class_weight='balanced', # VER SI CAMBIAR ESTE PARAMETRO
            C=2.0,                
            max_iter=10000
        ))
    ])
    
    print("Entrenando modelo OPTIMIZADO...")
    pipeline.fit(train_df[text_col], train_df[label_col])
    return pipeline
    


if 'train_data' in locals():
    # Preprocesar
    df_clean = prepare_data(train_data)

    # Split
    X_train, X_val, y_train, y_val = train_test_split(
         df_clean['text'], 
         df_clean['label_reduced'], 
         test_size=0.2, 
         stratify=df_clean['label_reduced'],
         random_state=42 # Recomendado para reproducibilidad
     )

    # Entrenar
    model = train_genre_classifier(pd.DataFrame({'text': X_train, 'label_reduced': y_train}))

    # Evaluar
    preds = model.predict(X_val)
    
    # CORRECCIÓN 2: Usar 'super_class_names'
    print(classification_report(y_val, preds, target_names=super_class_names))
else:
    print("⚠️ Error: La variable 'train_data' no está definida. Carga tu CSV primero.")

🚀 Iniciando búsqueda de hiperparámetros (GridSearch)...
Esto puede tardar unos minutos, pero encontrará la mejor configuración matemática.
Fitting 3 folds for each of 16 candidates, totalling 48 fits


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# 1. Definición del Mapeo
label_mapping = {
    "action": 0, "adventure": 0, "crime": 0, "fantasy": 0, "mystery": 0, "sci-fi": 0, "thriller": 0, "war": 0, "western": 0,
    "biography": 1, "drama": 1, "film-noir": 1, "history": 1, "romance": 1,
    "animation": 2, "comedy": 2, "family": 2, "music": 2, "musical": 2,
    "documentary": 3, "game-show": 3, "news": 3, "reality-tv": 3, "sport": 3, "talk-show": 3,
    "adult": 4, "horror": 4, "short": 4
}

super_class_names = ["Action_Suspense", "Drama_Serious", "Comedy_Family", "Factual_TV", "Niche_Horror"]

def prepare_data(df, text_col='text', label_col='genre'):
    df = df.copy()
    df['label_reduced'] = df[label_col].map(label_mapping)
    df = df.dropna(subset=['label_reduced'])
    df['label_reduced'] = df['label_reduced'].astype(int)
    return df

def train_genre_classifier(train_df, text_col='text', label_col='label_reduced'):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=20000,
            ngram_range=(1, 3),
            stop_words='english',
            min_df=2,
            sublinear_tf=True
        )),
        ('clf', LogisticRegression(
            multi_class='multinomial',
            solver='saga',
            class_weight='balanced', 
            C=1,
            max_iter=10000
        ))
    ])
    
    print("Entrenando modelo...")
    pipeline.fit(train_df[text_col], train_df[label_col])
    return pipeline

# --- BLOQUE DE EJECUCIÓN CORREGIDO ---

# Verificamos que tengas ambos datasets cargados
if 'train_data' in locals() and 'test_data' in locals():
    
    # 1. Preprocesar ambos
    print("Preprocesando datos...")
    df_train_clean = prepare_data(train_data)
    df_test_clean = prepare_data(test_data)

    # 2. Definir X e y para ENTRENAMIENTO
    X_train = df_train_clean['text']
    y_train = df_train_clean['label_reduced']

    # 3. Definir X e y para VALIDACIÓN (Usamos el test set)
    X_test = df_test_clean['text']
    y_test = df_test_clean['label_reduced']

    # 4. Entrenar (Pasamos los datos de train limpios)
    # Creamos un DF temporal porque tu función espera columnas con nombre
    train_subset = pd.DataFrame({'text': X_train, 'label_reduced': y_train})
    model = train_genre_classifier(train_subset)

    # 5. Evaluar (Usamos los datos de test)
    print("\nEvaluando modelo con datos de Test...")
    preds = model.predict(X_test)
    
    print(classification_report(y_test, preds, target_names=super_class_names))

elif 'train_data' in locals():
    # CASO: Si SOLO tienes train_data y no tienes test_data
    print("⚠️ No se detectó 'test_data', dividiendo train_data para validar...")
    df_clean = prepare_data(train_data)
    
    X_train, X_val, y_train, y_val = train_test_split(
         df_clean['text'], 
         df_clean['label_reduced'], 
         test_size=0.2, 
         stratify=df_clean['label_reduced'],
         random_state=42 
     )
    
    train_subset = pd.DataFrame({'text': X_train, 'label_reduced': y_train})
    model = train_genre_classifier(train_subset)
    
    preds = model.predict(X_val)
    print(classification_report(y_val, preds, target_names=super_class_names))

else:
    print("⚠️ Error: Carga tus CSVs en las variables 'train_data' y 'test_data' primero.")

Preprocesando datos...
Entrenando modelo...


c:\Users\alber\OneDrive\Desktop\Procesamiento del lenguaje natural II\Practica 1 NLP II\NLP_II_Practica1\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(



Evaluando modelo con datos de Test...
                 precision    recall  f1-score   support

Action_Suspense       0.70      0.86      0.77     29590
  Drama_Serious       0.58      0.51      0.54     11125
  Comedy_Family       0.49      0.25      0.33      4821
     Factual_TV       0.73      0.65      0.69      3609
   Niche_Horror       0.52      0.28      0.36      6255

       accuracy                           0.66     55400
      macro avg       0.60      0.51      0.54     55400
   weighted avg       0.64      0.66      0.64     55400



In [19]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.utils import resample # Importante para re-muestrear

# 1. Definición del Mapeo
label_mapping = {
    "action": 0, "adventure": 0, "crime": 0, "fantasy": 0, "mystery": 0, "sci-fi": 0, "thriller": 0, "war": 0, "western": 0,
    "biography": 1, "drama": 1, "film-noir": 1, "history": 1, "romance": 1,
    "animation": 2, "comedy": 2, "family": 2, "music": 2, "musical": 2,
    "documentary": 3, "game-show": 3, "news": 3, "reality-tv": 3, "sport": 3, "talk-show": 3,
    "adult": 4, "horror": 4, "short": 4
}

binary_class_names = ["Action_Suspense", "Drama_Serious"]

def prepare_data_binary(df, text_col='text', label_col='genre'):
    df = df.copy()
    df['label_reduced'] = df[label_col].map(label_mapping)
    # Filtro: Solo Action(0) y Drama(1)
    df = df[df['label_reduced'].isin([0, 1])]
    df['label_reduced'] = df['label_reduced'].astype(int)
    return df

def force_exact_balance(df, label_col='label_reduced'):
    """
    Esta función recorta la clase mayoritaria para que tenga 
    EXACTAMENTE el mismo número de filas que la minoritaria.
    """
    # Separamos los dos grupos
    df_action = df[df[label_col] == 0]
    df_drama = df[df[label_col] == 1]
    
    # Contamos cuántos hay de cada uno
    n_action = len(df_action)
    n_drama = len(df_drama)
    
    # Buscamos el mínimo (normalmente será Drama)
    min_samples = min(n_action, n_drama)
    
    print(f"   Original -> Action: {n_action}, Drama: {n_drama}")
    
    # Recortamos ambos al tamaño del más pequeño (Undersampling)
    df_action_down = df_action.sample(n=min_samples, random_state=42)
    df_drama_down = df_drama.sample(n=min_samples, random_state=42) # Si es el menor, se queda igual
    
    # Juntamos y mezclamos (shuffle)
    df_balanced = pd.concat([df_action_down, df_drama_down])
    df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"   BALANCEADO -> Action: {len(df_action_down)}, Drama: {len(df_drama_down)}")
    
    return df_balanced

def train_binary_classifier(train_df, text_col='text', label_col='label_reduced'):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(
            max_features=20000,
            ngram_range=(1, 2),
            stop_words='english',
            min_df=2,
            sublinear_tf=True
        )),
        ('clf', LogisticRegression(
            solver='saga',
            # YA NO USAMOS 'balanced' AQUÍ. 
            # Los datos entran 50/50, así que dejamos que el modelo trate a todos por igual.
            C=0.5,
            max_iter=10000
        ))
    ])
    
    print("Entrenando modelo...")
    pipeline.fit(train_df[text_col], train_df[label_col])
    return pipeline

# --- EJECUCIÓN ---

if 'train_data' in locals() and 'test_data' in locals():
    print("1. Preparando datos...")
    # Limpiamos (Aquí todavía están desbalanceados)
    df_train_raw = prepare_data_binary(train_data)
    df_test_raw = prepare_data_binary(test_data)

    # 2. BALANCEO MANUAL SOLO EN TRAIN
    print("2. Aplicando balanceo 50/50 al set de ENTRENAMIENTO...")
    df_train_balanced = force_exact_balance(df_train_raw)

    # El Test NO se toca (queremos ver la realidad)
    X_test = df_test_raw['text']
    y_test = df_test_raw['label_reduced']
    
    # 3. Entrenar
    model = train_binary_classifier(df_train_balanced)

    # 4. Evaluar
    print("\n--- RESULTADOS BINARIOS (Entrenamiento 50/50 estricto) ---")
    preds = model.predict(X_test)
    print(classification_report(y_test, preds, target_names=binary_class_names))

elif 'train_data' in locals():
    # Caso solo train: Dividimos primero, y luego balanceamos SOLO la parte de train
    df_raw = prepare_data_binary(train_data)
    
    X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
         df_raw['text'], 
         df_raw['label_reduced'], 
         test_size=0.2, 
         stratify=df_raw['label_reduced'],
         random_state=42 
     )
    
    # Reconstruimos el DF de train para pasarlo por la balanceadora
    df_train_temp = pd.DataFrame({'text': X_train_split, 'label_reduced': y_train_split})
    
    print("Aplicando balanceo 50/50 al split de ENTRENAMIENTO...")
    df_train_balanced = force_exact_balance(df_train_temp)
    
    model = train_binary_classifier(df_train_balanced)
    
    preds = model.predict(X_val_split)
    print("\n--- RESULTADOS BINARIOS (Entrenamiento 50/50 estricto) ---")
    print(classification_report(y_val_split, preds, target_names=binary_class_names))

else:
    print("⚠️ Error: Carga tus datos.")

1. Preparando datos...
2. Aplicando balanceo 50/50 al set de ENTRENAMIENTO...
   Original -> Action: 118362, Drama: 44497
   BALANCEADO -> Action: 44497, Drama: 44497
Entrenando modelo...

--- RESULTADOS BINARIOS (Entrenamiento 50/50 estricto) ---
                 precision    recall  f1-score   support

Action_Suspense       0.91      0.79      0.84     29590
  Drama_Serious       0.58      0.78      0.67     11125

       accuracy                           0.79     40715
      macro avg       0.74      0.78      0.75     40715
   weighted avg       0.82      0.79      0.79     40715

